Modeling: Mass Total + Source Inversion
=======================================

This script fits `Interferometer` dataset of a galaxy with a model where:

 - The galaxy's light is modeled using an `Inversion` with a rectangular pixelization and constant regularization
 scheme.

An `Inversion` reconstructs the source's light using a pixel-grid, which is regularized using a prior that forces
this reconstruction to be smooth.

For interferometer datasets, an inversion offers a compelling way to reconstruct the uv-plane data in real-space using
a pixel-grid which can account for irregular, asymmetric and clump galaxy morphologies. Thus, it allows one to
determine whether the uv-plane data has star forming clumps, which are not due to correlated noise inducated by
a cleaning algorithm.

Inversions are covered in detail in chapter 4 of the **HowToGalaxy** lectures.

__Positive Only Solver__

All pixelized source reconstructions use a positive-only solver, meaning that every source-pixel is only allowed
to reconstruct positive flux values. This ensures that the source reconstruction is physical and that we don't
reconstruct negative flux values that don't exist in the real source galaxy (a common systematic solution in lens
analysis).

It may be surprising to hear that this is a feature worth pointing out, but it turns out setting up the linear algebra
to enforce positive reconstructions is difficult to make efficient. A lot of development time went into making this
possible, where a bespoke fast non-negative linear solver was developed to achieve this.

Other methods in the literature often do not use a positive only solver, and therefore suffer from these
unphysical solutions, which can degrade the results of lens model in general.

In [ ]:
%matplotlib inline
from pyprojroot import here
workspace_path = str(here())
%cd $workspace_path
print(f"Working Directory has been set to `{workspace_path}`")

from os import path
import autofit as af
import autogalaxy as ag
import autogalaxy.plot as aplt
import numpy as np

__Mask__

We define the ‘real_space_mask’ which defines the grid the image the galaxy is evaluated using.

In [ ]:
real_space_mask = ag.Mask2D.circular(
    shape_native=(800, 800), pixel_scales=0.05, radius=3.0, sub_size=1
)

__Dataset__

Load and plot the galaxy `Interferometer` dataset `simple__sersic` from .fits files , which we will fit 
with the model.

In [ ]:
dataset_name = "simple__sersic"
dataset_path = path.join("dataset", "interferometer", dataset_name)

dataset = ag.Interferometer.from_fits(
    data_path=path.join(dataset_path, "data.fits"),
    noise_map_path=path.join(dataset_path, "noise_map.fits"),
    uv_wavelengths_path=path.join(dataset_path, "uv_wavelengths.fits"),
    real_space_mask=real_space_mask,
)

__Inversion Settings (Run Times)__

The run times of an interferometer `Inversion` depend significantly on the following settings:

 - `transformer_class`: whether a discrete Fourier transform (`TransformerDFT`) or non-uniform fast Fourier Transform
 (`TransformerNUFFT) is used to map the inversion's image from real-space to Fourier space.

 - `use_linear_operators`: whether the linear operator formalism or matrix formalism is used for the linear algebra.

The optimal settings depend on the number of visibilities in the dataset:

 - For N_visibilities < 1000: `transformer_class=TransformerDFT` and `use_linear_operators=False` gives the fastest
 run-times.
 - For  N_visibilities > ~10000: use `transformer_class=TransformerNUFFT`  and `use_linear_operators=True`.

The dataset modeled by default in this script has just 200 visibilties, therefore `transformer_class=TransformerDFT`
and `use_linear_operators=False`. If you are using this script to model your own dataset with a different number of
visibilities, you should update the options below accordingly.

The script `autogalaxy_workspace/*/interferometer/profiling.py` allows you to compute the run-time of an inversion
for your interferometer dataset. It does this for all possible combinations of settings and therefore can tell you
which settings give the fastest run times for your dataset.

In [ ]:
settings_dataset = ag.SettingsInterferometer(transformer_class=ag.TransformerDFT)
settings_inversion = ag.SettingsInversion(use_linear_operators=False)

We now create the `Interferometer` object which is used to fit the model.

This includes a `SettingsInterferometer`, which includes the method used to Fourier transform the real-space 
image of the galaxy to the uv-plane and compare directly to the visiblities. We use a non-uniform fast Fourier 
transform, which is the most efficient method for interferometer datasets containing ~1-10 million visibilities.

In [ ]:
dataset = dataset.apply_settings(settings=settings_dataset)
dataset_plotter = aplt.InterferometerPlotter(dataset=dataset)
dataset_plotter.subplot_dataset()
dataset_plotter.subplot_dirty_images()

__Model__

We compose our model using `Model` objects, which represent the galaxies we fit to our data.  In this 
example we fit a model where:

 - The galaxy's light uses a `Rectangular` mesh with fixed resolution 30 x 30 pixels (0 parameters). 
 - This pixelization is regularized using a `Constant` scheme which smooths every source pixel equally [1 parameter]. 

The number of free parameters and therefore the dimensionality of non-linear parameter space is N=1. 
 
It is worth noting the `Pixelization`  use significantly fewer parameters (1 parameter) than 
fitting the galaxy using parametric light profiles (7+ parameters). 

In [ ]:
pixelization = af.Model(
    ag.Pixelization,
    mesh=ag.mesh.Rectangular(shape=(30, 30)),
    regularization=ag.reg.Constant,
)

galaxy = af.Model(ag.Galaxy, redshift=0.5, pixelization=pixelization)

model = af.Collection(galaxies=af.Collection(galaxy=galaxy))

The `info` attribute shows the model in a readable format.

In [ ]:
print(model.info)

__Search__

The model is fitted to the data using a non-linear search. In this example, we use the nested sampling algorithm 
Nautilus (https://nautilus.readthedocs.io/en/latest/).

A full description of the settings below is given in the beginner modeling scripts, if anything is unclear.

In [ ]:
search = af.Nautilus(
    path_prefix=path.join("interferometer", "modeling"),
    name="light[pixelization]",
    unique_tag=dataset_name,
    n_live=100,
    number_of_cores=1,
)

__Analysis__

The `AnalysisInterferometer` object defines the `log_likelihood_function` used by the non-linear search to fit the 
model to the `Interferometer`dataset.

In [ ]:
analysis = ag.AnalysisInterferometer(
    dataset=dataset, settings_inversion=settings_inversion
)

__Model-Fit__

We can now begin the model-fit by passing the model and analysis object to the search, which performs a non-linear
search to find which models fit the data with the highest likelihood.

Checkout the output folder for live outputs of the results of the fit, including on-the-fly visualization of the best 
fit model!

In [ ]:
result = search.fit(model=model, analysis=analysis)


__Result__

The search returns a result object, which whose `info` attribute shows the result in a readable format:

In [ ]:
print(result.info)

The `Result` object also contains:

 - The model corresponding to the maximum log likelihood solution in parameter space.
 - The corresponding maximum log likelihood `Plane` and `FitImaging` objects.
 - Information on the posterior as estimated by the `Nautilus` non-linear search. 

In [ ]:
print(result.max_log_likelihood_instance)

plane_plotter = aplt.PlanePlotter(
    plane=result.max_log_likelihood_plane,
    grid=real_space_mask.derive_grid.unmasked_sub_1,
)
plane_plotter.subplot()

fit_plotter = aplt.FitInterferometerPlotter(fit=result.max_log_likelihood_fit)
fit_plotter.subplot_fit()
fit_plotter.subplot_fit_dirty_images()

search_plotter = aplt.DynestyPlotter(samples=result.samples)
search_plotter.cornerplot()

Checkout `autogalaxy_workspace/*/interferometer/modeling/results.py` for a full description of the result object.